In [ ]:
from pptx import Presentation
prs = Presentation('../private/casestudies.pptx')

slides = prs.slides
#slides = [prs.slides[i] for i in range(2)]
slide_text_array = []
for slide in slides:
    slide_text = ""
    shapes = slide.shapes
    for shape in shapes:
        if shape.has_text_frame:
            slide_text += shape.text + "\n"
#            print(shape.text)
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    address = run.hyperlink.address
                    if address is None:
                        continue
#                    print(address)
                    slide_text += address + "\n"
    slide_text_array.append(slide_text)

print(slide_text_array[0])



In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('AOAI_API_KEY')
openai.api_type = "azure"
openai.api_base = os.getenv('AOAI_ENDPOINT')
openai.api_version = "2023-05-15"

def get_completion_from_messages(messages,temperature=0):
    response = openai.ChatCompletion.create(
        engine=os.getenv('AOAI_MODEL'),
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        timeout=60
    )

    print(response.usage)

    return response.choices[0].message["content"]

In [ ]:
system_prompt = """あなたは事例まとめ作成AIサービスです。入力される事例情報をもとに、事例まとめを作成します。
なお、事例まとめには以下の要素を含め、マークダウン形式で出力します。

# 事例のタイトル
## 顧客名
## 利用したサービス
## 利用したモデル
## 何をどう解決したか
## 事例詳細のURL


出力例)
# 事例のタイトル
Azure OpenAI Serviceを利用した台湾の大学が英語学習プラットフォームを開発

## 顧客名
National Taiwan Normal University（国立台湾師範大学）

## 利用したサービス
- Azure OpenAI Service
- Azure AI
- Azure Cognitive Services

## 利用したモデル
Davinci advanced language model

## 何をどう解決したか
- 台湾の学生がグローバルな競争社会で成功するために英語のスピーキングスキルを向上させる支援
- リアルな会話やライティングの練習の機会が限られている課題を克服し、学生が学習目標を達成する手助け

## 事例詳細のURL
[事例詳細はこちら](https://customers.microsoft.com/en-US/story/1623125651482069014-national-taiwan-normal-university-higher-education-azure-openai-service)
"""

In [ ]:
system_prompt = """あなたはJSON作成機です。入力された事例情報をもとに以下の内容を含むJSONを作成します。
JSONには次の要素を含みます。1) company_name, 2) country, 3) services, 4) models, 5) what_they_deployed, 6) type_of_application, 7) url

説明や補足は不要です。
JSONのみ出力してください。

# 出力例
{
    "company_name": "国立台湾師範大学",
    "country": "台湾",
    "services": [
        "Azure OpenAI Service",
        "Azure AI",
        "Azure Cognitive Services"
    ],
    "models": [
        "Davinci advanced language model"
    ],
    "what_they_deployed": [
        "英語のスピーキングスキルを向上させるBotサービスを学生に提供"
    ],
    "type_of_application": "Bot application",
    "url": "https://customers.microsoft.com/en-US/story/1623125651482069014-national-taiwan-normal-university-higher-education-azure-openai-service"   
}
"""

In [ ]:
def getSummary(casestudy):
    messages =  [
    {'role':'system', 'content':system_prompt},
    {'role':'user', 'content':casestudy},
    ]
    response = get_completion_from_messages(messages, temperature=0)
    print(response)
    return response

In [ ]:
# slide_text_arrayループ
summary_json = []
for slide_text in slide_text_array:
    print(slide_text)
    # try-catchで例外が出たら再度処理する
    try:
        summary_json.append(getSummary(slide_text))
    except:
        print('ERROR! retry once')
        summary_json.append(getSummary(slide_text))
